In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from langchain_openai import ChatOpenAI

chatModel = ChatOpenAI(model="gpt-3.5-turbo-0125")

# RunnablePassthrough

* It does not do anything to the input data.

* Let’s see it in a very simple example: a chain with just RunnablePassthrough() will output the original input without any modification.


In [3]:

from langchain_core.runnables import RunnablePassthrough

chain = RunnablePassthrough()

In [4]:
chain.invoke("Caner")


'Caner'

# RunnableLambda

* To use a custom function inside a LCEL chain we need to wrap it up with RunnableLambda.

* Let's define a very simple function to create Russian lastnames:


In [5]:
def russian_lastname(name: str) -> str:
    return f"{name}ovich"

In [6]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(russian_lastname)

In [7]:
chain.invoke("Caner")


'Canerovich'

# RunnableParallel

* We will use RunnableParallel() for running tasks in parallel.
* This is probably the most important and most useful Runnable from LangChain.
* In the following chain, RunnableParallel is going to run these two tasks in parallel:
   * operation_a will use RunnablePassthrough.
   * operation_b will use RunnableLambda with the russian_lastname function.

In [8]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(russian_lastname)
    }
)

In [9]:
chain.invoke("Caner")


{'operation_a': 'Caner', 'operation_b': 'Canerovich'}

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

In [12]:
prompt = ChatPromptTemplate.from_template("tell me a curious fact about {Football_player}")

output_parser = StrOutputParser()

In [13]:
def russian_lastname_from_dictionary(person):
    return person["name"] + "ovich"

In [16]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "Football_player": RunnableLambda(russian_lastname_from_dictionary),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | chatModel | output_parser

In [18]:
chain.invoke({
    "name1": "Jordan",
    "name": "Abram"
})

'One curious fact about Roman Abramovich is that he was once the governor of the remote Chukotka region in Russia, where he invested millions of dollars into infrastructure, education, and healthcare for the local population. Abramovich was known for personally visiting the region and participating in various community projects during his time as governor.'

#  More advanced use of RunnableParallel


In [20]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

vectorstore = FAISS.from_texts(
    ["Caner has interest to  Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI(model="gpt-3.5-turbo")

retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | model
    | StrOutputParser()
)

retrieval_chain.invoke("What is Caner?")

'Caner is interested in Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English.'

In [24]:
from operator import itemgetter

from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

model = ChatOpenAI(model="gpt-3.5-turbo")

vectorstore = FAISS.from_texts(
    ["Caner has interest to  Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English."], embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language"),
    }
    | prompt
    | model
    | StrOutputParser()
)

chain.invoke({"question": "What is caner?", "language": "Pirate English"})

'Arrr! Caner be a scallywag with interest in Data Science, AI, ML, DL, CV, NLP, Python programming, etc. in English. Arrr!'

# Important: the syntax of RunnableParallel can have several variations.

When composing a RunnableParallel with another Runnable you do not need to wrap it up in the RunnableParallel class. Inside a chain, the next three syntaxes are equivalent:

RunnableParallel({"context": retriever, "question": RunnablePassthrough()})

RunnableParallel(context=retriever, question=RunnablePassthrough())

{"context": retriever, "question": RunnablePassthrough()}
